# 8. 최적화 기법 및 고급 주제

이 챕터에서는 추천 시스템의 성능을 더욱 향상시키기 위한 다양한 최적화 기법과 고급 주제를 다룹니다. 메모리 효율성, 계산 속도 개선, 하이브리드 접근법, 그리고 딥러닝 기반 추천 시스템에 대한 내용을 포함합니다.

## 8.1 희소 행렬을 활용한 메모리 최적화

추천 시스템에서는 사용자-아이템 행렬이 매우 크고 대부분의 값이 0인 희소 행렬(Sparse Matrix)인 경우가 많습니다. 이 경우 scipy의 희소 행렬 구현을 활용하여 메모리 사용량과 계산 효율성을 크게 개선할 수 있습니다.

### CSR 행렬(Compressed Sparse Row Matrix)

CSR은 희소 행렬을 효율적으로 저장하고 계산하기 위한 표준 데이터 구조입니다.

**작동 원리:**
1. **데이터 배열**: 0이 아닌 값들만 저장
2. **인덱스 배열**: 각 값의 열 인덱스를 저장
3. **포인터 배열**: 각 행의 시작 위치를 가리키는 포인터 저장

**장점:**
- **메모리 효율성**: 0이 아닌 값만 저장하여 메모리 사용량 대폭 감소
- **계산 효율성**: 행렬 연산(곱셈, 덧셈 등)이 0이 아닌 요소에만 적용되어 계산 속도 향상
- **행 접근 최적화**: 특정 행 접근이 빠름(열 접근은 상대적으로 느림)

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import time
from collections import defaultdict
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

# 상위 디렉토리 경로를 시스템 경로에 추가하여 utils 모듈을 import할 수 있게 함
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# utils 모듈에서 필요한 클래스와 함수를 import
from utils.models import Dataset, RecommendResult, Metrics
from utils.data_loader import DataLoader
from utils.metric_calculator import MetricCalculator

### 희소 행렬을 활용한 UserKNN 구현

In [ ]:
class SparseUserKNNRecommender(BaseRecommender):
    """희소 행렬을 활용한 사용자 기반 협업 필터링
    
    CSR 행렬을 사용하여 메모리 효율성과 계산 속도를 개선한 UserKNN 구현
    """
    def recommend(self, dataset: Dataset, k=10, n_neighbors=20, **kwargs) -> RecommendResult:
        train, test = dataset.train, dataset.test
        mu = float(train['rating'].mean())  # 전체 평균 평점
        
        # 사용자-아이템 행렬 생성 (희소 행렬 사용)
        user_indices = {u: i for i, u in enumerate(train.userId.unique())}
        item_indices = {m: j for j, m in enumerate(train.movieId.unique())}
        
        n_users = len(user_indices)
        n_items = len(item_indices)
        
        # CSR 행렬 생성을 위한 데이터 준비
        row = [user_indices[u] for u in train.userId.values]
        col = [item_indices[m] for m in train.movieId.values]
        data = train.rating.values
        
        # CSR 행렬 생성
        ui_sparse = csr_matrix((data, (row, col)), shape=(n_users, n_items))
        
        # 유사도 행렬 계산 (배치 처리로 메모리 관리)
        batch_size = 1000  # 메모리에 따라 조정
        sim = np.zeros((n_users, n_users))
        
        for i in range(0, n_users, batch_size):
            end = min(i + batch_size, n_users)
            batch_sim = cosine_similarity(ui_sparse[i:end], ui_sparse)
            sim[i:end] = batch_sim
        
        # 역방향 매핑 생성
        idx_to_user = {i: u for u, i in user_indices.items()}
        idx_to_item = {j: m for m, j in item_indices.items()}
        
        # 사용자별 이미 평가한 아이템 목록
        seen = train.groupby('userId')['movieId'].apply(set).to_dict()
        
        # RMSE 계산을 위한 평점 예측
        pred = pd.Series(index=test.index, dtype=float)
        for idx, row in test.iterrows():
            u, m = row['userId'], row['movieId']
            
            # 학습 데이터에 없는 사용자나 아이템 처리
            if u not in user_indices or m not in item_indices:
                pred.at[idx] = mu
                continue
                
            u_idx = user_indices[u]
            m_idx = item_indices[m]
            
            # 이웃 선택
            user_similarities = sim[u_idx]
            neighbors = np.argsort(user_similarities)[::-1][1:n_neighbors+1]  # 자기 자신 제외
            
            # 이웃의 해당 아이템 평점 얻기
            neighbor_ratings = []
            neighbor_sims = []
            
            for n_idx in neighbors:
                rating = ui_sparse[n_idx, m_idx]
                if rating > 0:  # 평점이 있는 경우만
                    neighbor_ratings.append(rating)
                    neighbor_sims.append(user_similarities[n_idx])
            
            # 가중 평균 계산
            if neighbor_ratings:
                weights_sum = sum(neighbor_sims)
                if weights_sum > 1e-8:
                    pred.at[idx] = sum(r*s for r, s in zip(neighbor_ratings, neighbor_sims)) / weights_sum
                else:
                    pred.at[idx] = mu
            else:
                pred.at[idx] = mu
                
        # 예측값 범위 조정
        pred = pred.clip(lower=0.5, upper=5.0)
        
        # 추천 목록 생성 (간소화된 버전)
        user2items = defaultdict(list)
        for u in test['userId'].unique():
            if u not in user_indices:
                continue
                
            u_idx = user_indices[u]
            u_seen = seen.get(u, set())
            
            # 사용자와 유사한 이웃 선택
            neighbors = np.argsort(sim[u_idx])[::-1][1:n_neighbors+1]
            
            # 아이템 점수 계산
            item_scores = defaultdict(float)
            for n_idx in neighbors:
                n_sim = sim[u_idx, n_idx]
                n_items = ui_sparse[n_idx].nonzero()[1]  # 이웃이 평가한 아이템
                
                for i_idx in n_items:
                    item_id = idx_to_item[i_idx]
                    if item_id not in u_seen:  # 이미 평가한 아이템 제외
                        rating = ui_sparse[n_idx, i_idx]
                        item_scores[item_id] += n_sim * rating
            
            # 점수가 높은 순서로 상위 k개 아이템 추천
            sorted_items = sorted(item_scores.items(), key=lambda x: -x[1])[:k]
            user2items[u] = [item_id for item_id, _ in sorted_items]
            
        return RecommendResult(pred, user2items)

### 성능 비교: 일반 구현 vs 희소 행렬 구현

In [ ]:
# 데이터셋 로드
loader = DataLoader(num_users=1000)  # 더 많은 사용자로 테스트
dataset = loader.load()

# 일반 UserKNN 실행 시간 측정
start_time = time.time()
normal_recommender = UserKNNRecommender()
normal_result = normal_recommender.recommend(dataset, k=10, n_neighbors=20)
normal_time = time.time() - start_time

# 희소 행렬 UserKNN 실행 시간 측정
start_time = time.time()
sparse_recommender = SparseUserKNNRecommender()
sparse_result = sparse_recommender.recommend(dataset, k=10, n_neighbors=20)
sparse_time = time.time() - start_time

# 결과 출력
print(f"일반 UserKNN 실행 시간: {normal_time:.2f}초")
print(f"희소 행렬 UserKNN 실행 시간: {sparse_time:.2f}초")
print(f"속도 향상: {normal_time/sparse_time:.2f}배")

## 8.2 하이브리드 추천 시스템

하이브리드 추천 시스템은 여러 추천 알고리즘을 결합하여 각 알고리즘의 강점을 활용하고 약점을 보완하는 접근법입니다. 여기서는 평점 예측에 강한 MF와 추천 품질에 우수한 ItemKNN을 결합한 하이브리드 모델을 구현해 봅니다.

In [ ]:
class HybridRecommender(BaseRecommender):
    """MF와 ItemKNN을 결합한 하이브리드 추천 시스템
    
    평점 예측에 강한 MF와 추천 품질이 우수한 ItemKNN을 결합하여
    RMSE와 Precision/Recall 모두 우수한 성능을 제공합니다.
    """
    def recommend(self, dataset: Dataset, k=10, alpha=0.7, **kwargs) -> RecommendResult:
        # MF 모델 학습 및 예측
        mf = MFRecommender()
        mf_result = mf.recommend(
            dataset, 
            k=k, 
            n_factors=kwargs.get('n_factors', 20),
            learning_rate=kwargs.get('learning_rate', 0.02),
            n_epochs=kwargs.get('n_epochs', 30),
            reg=kwargs.get('reg', 0.08)
        )
        
        # ItemKNN 모델 학습 및 예측
        item_knn = ItemKNNRecommender()
        item_knn_result = item_knn.recommend(
            dataset,
            k=k,
            n_neighbors=kwargs.get('n_neighbors', 10)
        )
        
        # 평점 예측: MF와 ItemKNN의 가중 평균
        mf_ratings = mf_result.rating
        item_knn_ratings = item_knn_result.rating
        
        # 두 모델의 예측값 결합 (가중 평균)
        hybrid_ratings = alpha * mf_ratings + (1 - alpha) * item_knn_ratings
        
        # 추천 목록: 두 모델의 점수 결합하여 새로운 추천 목록 생성
        train = dataset.train
        test = dataset.test
        seen = train.groupby('userId')['movieId'].apply(set).to_dict()
        
        # 두 모델의 추천 목록 합치기
        hybrid_user2items = defaultdict(list)
        test_users = test['userId'].unique()
        
        for u in test_users:
            # 이미 평가한 아이템 제외
            u_seen = seen.get(u, set())
            
            # 두 모델에서 추천된 아이템과 점수 수집
            item_scores = defaultdict(float)
            
            # MF 추천 목록
            if u in mf_result.user2items:
                mf_items = mf_result.user2items[u]
                for i, item in enumerate(mf_items):
                    if item not in u_seen:
                        # 순위에 따른 가중치 (역순으로 더 높은 점수)
                        item_scores[item] += alpha * (len(mf_items) - i)
            
            # ItemKNN 추천 목록
            if u in item_knn_result.user2items:
                item_knn_items = item_knn_result.user2items[u]
                for i, item in enumerate(item_knn_items):
                    if item not in u_seen:
                        # 순위에 따른 가중치
                        item_scores[item] += (1 - alpha) * (len(item_knn_items) - i)
            
            # 점수가 높은 순서로 정렬하여 상위 k개 추천
            sorted_items = sorted(item_scores.items(), key=lambda x: -x[1])[:k]
            hybrid_user2items[u] = [item for item, _ in sorted_items]
        
        return RecommendResult(hybrid_ratings, hybrid_user2items)

### 하이브리드 모델 평가

In [ ]:
# 데이터셋 로드
loader = DataLoader(num_users=100)
dataset = loader.load()

# 하이브리드 추천 시스템 평가
print("\n=== 하이브리드 추천 시스템 평가 ===")

# 다양한 alpha 값으로 실험
for alpha in [0.3, 0.5, 0.7]:
    hybrid_recommender = HybridRecommender()
    hybrid_result = hybrid_recommender.recommend(
        dataset,
        k=10,
        alpha=alpha,
        n_factors=20,
        n_neighbors=10
    )
    
    # 평가 지표 계산
    mc = MetricCalculator()
    metrics = mc.calc(
        dataset.test['rating'].tolist(),
        hybrid_result.rating.tolist(),
        dataset.test_user2items,
        hybrid_result.user2items,
        k=10,
        params={"model": "Hybrid", "alpha": alpha}
    )
    
    print(f"Hybrid (alpha={alpha}):", metrics)

## 8.3 효율적인 Top-K 추천 생성 기법

현실적인 추천 시스템에서는 모든 사용자-아이템 쌍에 대해 평점을 예측하는 것이 비효율적입니다. 특히 대규모 시스템에서는 Top-K 추천 목록만 효율적으로 생성하는 방법이 필요합니다.

In [ ]:
class EfficientMFRecommender(BaseRecommender):
    """효율적인 Top-K 추천을 위해 최적화된 MF 추천 시스템
    
    모든 아이템에 대한 평점을 계산하지 않고, 후보 아이템 집합을 먼저 구성하여
    계산량을 줄인 효율적인 Matrix Factorization 구현
    """
    def recommend(self, dataset: Dataset, k=10, **kwargs) -> RecommendResult:
        # 기존 MF 모델과 동일한 학습 과정...
        np.random.seed(0)
        n_factors = kwargs.get('n_factors', 20)
        lr = kwargs.get('learning_rate', 0.01)
        n_epochs = kwargs.get('n_epochs', 30)
        reg = kwargs.get('reg', 0.08)
        max_candidates = kwargs.get('max_candidates', 100)  # 후보 아이템 수 제한

        train, test = dataset.train, dataset.test
        mu = float(train['rating'].mean())

        users = sorted(train.userId.unique())
        items = sorted(train.movieId.unique())
        uid2i = {u:i for i,u in enumerate(users)}
        iid2j = {m:j for j,m in enumerate(items)}
        nU, nI = len(users), len(items)

        P = 0.1*np.random.randn(nU, n_factors)
        Q = 0.1*np.random.randn(nI, n_factors)
        bu = np.zeros(nU); bi = np.zeros(nI)

        df = train.copy()
        df['ui'] = df['userId'].map(uid2i)
        df['ij'] = df['movieId'].map(iid2j)

        # SGD 학습...
        for _ in range(n_epochs):
            for r in df.itertuples(index=False):
                u, j, y = int(r.ui), int(r.ij), float(r.rating)
                pred = mu + bu[u] + bi[j] + P[u].dot(Q[j])
                e = y - pred
                bu[u] += lr*(e - reg*bu[u])
                bi[j] += lr*(e - reg*bi[j])
                Pu = P[u].copy(); Qj = Q[j].copy()
                P[u] += lr*(e*Qj - reg*Pu)
                Q[j] += lr*(e*Pu - reg*Qj)

        # 테스트 데이터 RMSE용 예측 (변경 없음)
        pred = pd.Series(index=test.index, dtype=float)
        for idx, row in test.iterrows():
            u, m = row['userId'], row['movieId']
            if (u not in uid2i) or (m not in iid2j):
                pred.at[idx] = mu
            else:
                ui = uid2i[u]; ij = iid2j[m]
                pred.at[idx] = mu + bu[ui] + bi[ij] + P[ui].dot(Q[ij])
        pred = pred.clip(lower=0.5, upper=5.0)

        # 효율적인 Top-K 추천 목록 생성
        seen = train.groupby('userId')['movieId'].apply(set).to_dict()
        popular_items = train.groupby('movieId')['rating'].count().sort_values(ascending=False)
        popular_items = [mid for mid in popular_items.index if mid in iid2j]
        
        user2items = defaultdict(list)
        for u in test['userId'].unique():
            if u not in uid2i:
                continue
                
            ui = uid2i[u]
            u_seen = seen.get(u, set())
            
            # 아이템 후보군 선정 (인기도 기준으로 일부만 선택)
            candidate_items = [m for m in popular_items if m not in u_seen][:max_candidates]
            
            # 선정된 후보 아이템에 대해서만 점수 계산
            scores = {}
            for m in candidate_items:
                ij = iid2j[m]
                scores[m] = mu + bu[ui] + bi[ij] + P[ui].dot(Q[ij])
                
            # 점수가 높은 순으로 상위 k개 추천
            user2items[u] = _recommend_sorted(scores, u_seen, k)
            
        return RecommendResult(pred, user2items)

## 8.4 향후 발전 방향: 딥러닝 기반 추천 시스템

최근 추천 시스템 분야에서는 딥러닝을 활용한 다양한 접근법이 연구되고 있습니다. 이러한 모델은 전통적인 협업 필터링보다 복잡한 패턴을 학습할 수 있어 더 정확한 추천이 가능합니다. 주요 딥러닝 기반 추천 시스템 모델은 다음과 같습니다:

### Neural Collaborative Filtering (NCF)

NCF는 전통적인 행렬 분해를 신경망으로 확장한 모델입니다. 사용자와 아이템 임베딩을 다층 신경망에 통과시켜 비선형 상호작용을 모델링합니다.

주요 특징:
- MLP(Multi-Layer Perceptron)를 통한 사용자-아이템 상호작용 학습
- 전통적 행렬 분해와 신경망 결과를 결합한 하이브리드 구조
- 암시적 피드백(클릭, 구매 등)을 활용한 추천 가능

### Deep FM (Factorization Machine)

Deep FM은 팩터라이제이션 머신과 딥러닝을 결합한 모델로, 저차원 및 고차원 특성 상호작용을 모두 포착합니다.

주요 특징:
- FM 컴포넌트: 저차원 특성 상호작용 모델링
- DNN 컴포넌트: 고차원 특성 상호작용 모델링
- CTR(Click-Through Rate) 예측 등에 효과적

### Sequential Recommendation Models

사용자의 시간적 행동 패턴을 고려하는 순차적 추천 모델도 최근 주목받고 있습니다.

주요 모델:
- **GRU4Rec**: GRU(Gated Recurrent Unit) 기반 세션 중심 추천
- **SASRec**: Self-Attention 기반 순차 추천
- **BERT4Rec**: BERT와 같은 양방향 모델을 활용한 순차 추천

### Graph Neural Network 기반 추천

사용자와 아이템, 그리고 다른 엔티티(카테고리, 태그 등)를 노드로 하는 그래프에 GNN을 적용하는 접근법입니다.

주요 모델:
- **NGCF (Neural Graph Collaborative Filtering)**: 사용자-아이템 상호작용 그래프에 GNN 적용
- **LightGCN**: NGCF를 단순화하고 성능을 개선한 모델
- **KGAT (Knowledge Graph Attention Network)**: 지식 그래프와 추천을 결합한 모델

이러한 딥러닝 기반 모델은 더 복잡한 패턴을 학습할 수 있지만, 더 많은 데이터와 계산 리소스가 필요합니다. 실제 적용 시에는 데이터셋 특성, 비즈니스 요구사항, 그리고 가용한 리소스를 고려하여 적절한 모델을 선택해야 합니다.